# BM3D

Python implementation of ["An Analysis and Implementation of the BM3D Image Denoising Method" by Marc Lebrun](https://www.ipol.im/pub/art/2012/l-bm3d/?utm_source=doi).

## Modules

In [ ]:
import numpy as np
from skimage import io as skio
import pywt
import math
from scipy.fftpack import dct, idct

import matplotlib.pyplot as plt

## Parameters

In [ ]:
# 1st step
kHard = 8  #patch size
nHard = 39 #search window size --! era pra ser 39 mas nao entendi como centralizar P
NHard = 16 #max number of similar patches kept 
pHard = 3

sigma = 30
tauHard = 5000 if sigma > 40 else 2500

lambdaHard2d = 0 #hard thresholding for grouping --! ??? where
lambdaHard3d = 2.7

In [ ]:
# 2nd step
kWien = 8
nWien = 39
NWien = 32
pWien = 3

tauWien = 3500 if sigma > 40 else 400

## Initialization

In [ ]:
im = skio.imread('./lena.tif') # original image

In [ ]:
im = skio.imread('./muro.tif', as_gray=True) # original image

In [ ]:
im.shape

In [ ]:
def noise(im,br):
    """ Cette fonction ajoute un bruit blanc gaussier d'ecart type br
       a l'image im et renvoie le resultat"""
    imt=np.float32(im.copy())
    sh=imt.shape
    bruit=br*np.random.randn(*sh)
    imt=imt+bruit
    return imt

def noise2(im,br):
    """ Cette fonction ajoute un bruit blanc gaussier d'ecart type br
       a l'image im et renvoie le resultat"""
    imt=np.float32(im.copy())
    sh=imt.shape
    bruit=np.random.normal(0, br, sh)
    imt=imt+bruit
    #imt = np.clip(imt, 0, 1)
    return imt#np.float32(imt * 255)

In [ ]:
def normalize (image, vmin=0, vmax=255):
    image = (image - image.min()) / (image.max() - image.min()) * vmax
    image = np.clip(image, vmin, vmax).astype(np.float32)
    return image

In [ ]:
im = normalize(im)
imbr = noise2(im, sigma)

In [ ]:
plt.imshow(imbr, cmap='gray')

## The First Denoising Step


### Grouping

In [ ]:
# x,y is the top-left corner of the reference patch
# doesnt work well for even window_size --change
def get_search_window(image, x, y, patch_size=kHard, window_size=nHard):
    """ Given an image and the xy coordinates of the top-left corner of a reference patch
        Finds the search window with the reference patch in the center
        Returns the search window and the coordinates of its top-left pixel
    """
        
    window_top_left_x = x - (window_size//2 - patch_size//2)
    window_top_left_y = y - (window_size//2 - patch_size//2)
    
    search_window = image[
        window_top_left_x: window_top_left_x + window_size,
        window_top_left_y: window_top_left_y + window_size 
    ]
    return search_window, window_top_left_x, window_top_left_y

In [ ]:
def hard_thresholding(img, threshold):
    """ Applies a hard thresholding operation to an array
    """
    return (abs(img) > threshold) * img

In [ ]:
def grouping_1st_step(x, y, image, sigma, patch_size, window_size, lambdaHard2d, tauHard, N=NHard):
    # reference patch as array
    ref_patch = image[x:x+patch_size, y:y+patch_size]
    ref_patch_array = ref_patch.reshape(-1, patch_size**2)
    
    # vectorized patches from search window
    search_window, x_win, y_win = get_search_window(image, x, y, patch_size, window_size)
    window_patches = np.lib.stride_tricks.sliding_window_view(search_window, (patch_size, patch_size))
    window_patches_array = window_patches.reshape(-1, patch_size**2)
    
    # hard thresholding
    if sigma > 40:
        ref_patch_array = hard_thresholding(ref_patch_array, lambdaHard2d * sigma)
        window_patches_array = hard_thresholding(window_patches_array, lambdaHard2d * sigma)

    # calculate vector differences to reference patch
    diff_squared = (ref_patch_array - window_patches_array) **2
    ssd_array = np.sum(diff_squared, axis=1)
    dist_array = ssd_array / (kHard ** 2)
    
    # get N closest patches, N must be power of 2 and distance must be < tauHard
    N = 2 ** (math.floor(math.log2(N)))
    closer_indeces = dist_array.argsort()[:N]
    closer_indeces = np.array([i for i in closer_indeces if dist_array[i] < tauHard]) #apply similarity threshold
    
    size = len(closer_indeces)
    if not (size & (size-1) == 0):
        new_size = 2 ** (math.floor(math.log2(size)))
        closer_indeces = closer_indeces[:new_size]
    
    # get top left coord of each patch and build the 3d group
    closer_coords = np.array([[x_win+(i//(window_size - patch_size + 1)), y_win+(i%(window_size - patch_size + 1))] for i in closer_indeces])
    closer_patches = np.array([window_patches_array[i].reshape(patch_size, patch_size) for i in closer_indeces])

    return closer_patches, closer_coords

After the grouping in 1st step, we get a list of tuples formed by: 
- a numpy array (N, 8, 8) where 1 <= N <= NHard (16), which represent a 3d group
- the indeces (i, j) of the reference patch linked to that 3d group 

DÚVIDA: como lidar com patches que não formam grupos? acho que quando nenhum Q teve distancia abaixo da threshold. por enquanto vou só ignorar eles 

limitar N a uma potencia de 2 quando nao chega a 16

do we keep the reference patch in its 3d group?

### Collaborative Filtering

After the grouping in 1st step, we have a list of tuples formed by: 
- a numpy array (N, 8, 8) where 1 <= N <= NHard (16), which represent a 3d group
- the indeces (i, j) of the reference patch linked to that 3d group 

In the Collaborative Filtering step, we will perform the following steps for each 3d group:
1. apply a 3D isometric linear transform to the 3d block
2. apply a hard thresholding operator with threshold $\lambda ^{hard} _{3D} \sigma$
3. apply the inverse linear transform
4. save in a buffer the estimate found for each pixel in each patch (this will later be used in the aggregation step)

This means that the reference patch of size kHard x kHard (8x8) and top-left pixel in (18,489) should be estimated according to the 16 8x8 patches in the 3d group stored in a[1000][0]. 

In [ ]:
def walsh_hadamard_transform(x):
    """
    Apply the Walsh-Hadamard transform on a 1D array.
    
    Parameters:
    x (np.ndarray): 1D input array of length 2^n where n is a non-negative integer.
    
    Returns:
    np.ndarray: The Walsh-Hadamard transformed array.
    """
    n = len(x)
    if n & (n - 1) != 0:
        raise ValueError("Length of input array must be a power of 2")

    # Recursive base case
    if n == 1:
        return x
    
    # Split array into even and odd indexed parts
    even = walsh_hadamard_transform(x[0::2])
    odd = walsh_hadamard_transform(x[1::2])

    return np.concatenate([even + odd, even - odd]) 

In [ ]:
def get_Bior_matrices(N=8):
    directBior15_matrix=np.zeros((N,N))
    ss=N//2
    ls=[]

    while ss>0:
        ls=ls+[ss]
        ss=ss//2
    print (ls)   
    for k in range(N):
        inp=np.zeros(N)
        inp[k]=1
        tmp=inp
        out=[]
        for s in ls:
            #print (out,s)
            (a,b)=pywt.dwt(tmp,'bior1.5',mode='periodic')
            out=list(b[0:s])+out
            tmp=a[:s]
            #print ('sortie s=',s)
        out=list(a[:s])+out
        directBior15_matrix[k,:]=np.asarray(out)

    invBior15_matrix=np.linalg.inv(directBior15_matrix)
    return directBior15_matrix,invBior15_matrix

In [ ]:
def apply_bior(V,M,dim):
    s=V.shape
    l=[0,1,2]
    l[dim]=0
    l[0]=dim
    smod=list(s)
    smod[dim]=s[0]
    smod[0]=s[dim]
    return (M@V.transpose(l).reshape(((M.shape[0]),-1))).reshape(smod).transpose(l)

In [ ]:
def apply_1d_transform(array, use_dct=False):
    return walsh_hadamard_transform(array) / np.sqrt(len(array))

In [ ]:
def dct2d(block):
    return dct(dct(block, axis=0, norm='ortho'), axis=1, norm='ortho')

def idct2d(block):
    return idct(idct(block, axis=1, norm='ortho'), axis=0, norm='ortho')


In [ ]:
B8,IB8=get_Bior_matrices(N=8)
def apply_2d_transform(v, use_dct=False):
    if use_dct:
        return [dct2d(block) for block in v]
    else:
        v1d=apply_bior(v,B8,-1)
        v2d=apply_bior(v1d,B8,-2)
        return v2d

def reverse_2d_transform(v, use_dct=False):
    if use_dct:
        return [idct2d(block) for block in v]
    else:
        vappinv1d=apply_bior(v,IB8,-1)
        vappinv2d=apply_bior(vappinv1d,IB8,-2)
        return vappinv2d

### Aggregation

In [ ]:
def update_aggregation_buffers(nu, delta, patches, coords, weights, X, Y):
    weights = weights.reshape(-1,1,1)
    
    # Offset the grid for each patch position
    X_ = X + coords[:, 0].reshape(-1, 1, 1)  
    Y_ = Y + coords[:, 1].reshape(-1, 1, 1)  
    
    np.add.at(nu, (X_, Y_), patches * weights)
    np.add.at(delta, (X_, Y_), weights) #######
    
    return nu, delta

### 1st step

In [ ]:
def bm3d_1st_step(image, sigma, kHard, nHard, lambdaHard2d, lambdaHard3d, tauHard, NHard):
    height, width = image.shape

    # pad image and iterate through original frame
    window_size = nHard
    offset = window_size // 2
    padded_image = np.pad(image, offset, mode='reflect')

    nu = np.zeros(padded_image.shape)
    delta = np.zeros(padded_image.shape)

    X, Y = np.meshgrid(np.arange(kHard), np.arange(kHard), indexing='ij')
    
    # iterate through patches in the image with a step
    for x in range(offset, offset + height - kHard + 1, pHard):
        for y in range(offset , offset + width - kHard + 1, pHard):

            # GROUPING
            group3d, coords = grouping_1st_step(x, y, padded_image, sigma, patch_size=kHard, window_size=nHard, lambdaHard2d=lambdaHard2d, tauHard=tauHard, N=NHard)
            if len(coords) < 1:
                continue

            # COLLABORATIVE FILTERING
            # 3d transform
            transformed = np.array(apply_2d_transform(group3d, use_dct=False))
            transformed = apply_1d_transform(transformed) #t1d

            # thresholding
            threshold = lambdaHard3d * sigma
            thresholded = hard_thresholding(transformed, threshold)

            # calculate weights
            NPHard = np.count_nonzero(thresholded)
            weight = 1/NPHard if NPHard >= 1 else 1
            weight = np.array([weight])

            # 3d reverse
            thresholded = apply_1d_transform(thresholded) #t1d
            
            filtered = np.array(reverse_2d_transform(thresholded, use_dct=False))

            # AGGREGATION 
            nu, delta = update_aggregation_buffers(nu, delta, filtered, coords, weight, X, Y)

    ## BASIC ESTIMATE
    basic = np.divide(nu[offset:offset+height, offset:offset+width], delta[offset:offset+height, offset:offset+width])

    return basic, nu, delta

## The Second Denoising Step

In [ ]:
u_basic = bm3d_1st_step(imbr, sigma, kHard, nHard, lambdaHard2d, lambdaHard3d, tauHard, NHard)

### Grouping

In [ ]:
def grouping_2nd_step(x, y, image, basic, sigma, patch_size, window_size, lambdaHard2d, tauWien, NWien):
    #group formed by the basic estimate
    basic_patches, basic_coords = grouping_1st_step(x, y, basic, sigma, patch_size, window_size, lambdaHard2d, tauWien, NWien)

    #group formed by the original image
    original_patches = np.array([image[i:i+patch_size, j:j+patch_size] for i,j in basic_coords])

    return original_patches, basic_patches, basic_coords

### Collaborative Filtering


### Aggregation

In [ ]:
def bm3d_2nd_step(image, basic_estimate, sigma, kWien, nWien, lambdaHard2d, lambdaHard3d, tauWien, NWien):
    height, width = image.shape

    # pad image and iterate through original frame
    window_size = nWien
    offset = window_size // 2
    padded_image = np.pad(image, offset, mode='reflect')
    padded_basic = np.pad(basic_estimate, offset, mode='reflect')

    nu = np.zeros(padded_image.shape)
    delta = np.zeros(padded_image.shape)

    X, Y = np.meshgrid(np.arange(kWien), np.arange(kWien), indexing='ij')
    
    # iterate through patches in the image with a step
    for x in range(offset, offset + height - kWien + 1, pWien):
        for y in range(offset , offset + width - kWien + 1, pWien):

            # GROUPING
            group3d_original, group3d_basic, coords = grouping_2nd_step(x, y, padded_image, padded_basic, sigma, patch_size=kWien, window_size=nWien, lambdaHard2d=lambdaHard2d, tauWien=tauWien, NWien=NWien)
            if len(coords) < 1:
                continue

            # COLLABORATIVE FILTERING
            # basic 3D transform
            basic_transformed = np.array(apply_2d_transform(group3d_basic, use_dct=True))
            basic_transformed = apply_1d_transform(basic_transformed) 

            # wiener coefficients over basic
            module = np.absolute(basic_transformed) ** 2
            wp = module / (module + sigma**2) 

            # original 3D transform
            original_transformed = np.array(apply_2d_transform(group3d_original, use_dct=True))
            original_transformed = apply_1d_transform(original_transformed) 

            # wiener filtering over original
            original_filtered = wp * original_transformed

            # 3D reverse
            filtered = apply_1d_transform(original_filtered)
            filtered = np.array(reverse_2d_transform(filtered, use_dct=True))

            # calculate weights
            weight = np.array([np.linalg.norm(wp) ** (-2)])

            # AGGREGATION 
            nu, delta = update_aggregation_buffers(nu, delta, filtered, coords, weight, X, Y)

    ## FINAL ESTIMATE
    final = np.divide(nu[offset:offset+height, offset:offset+width], delta[offset:offset+height, offset:offset+width])

    return final, nu, delta

## Results

In [ ]:
plt.imshow(im, cmap='gray')

In [ ]:
plt.imshow(imbr, cmap='gray')

In [ ]:
%%time
basic_estimate = bm3d_1st_step(imbr, sigma, kHard, nHard, lambdaHard2d, lambdaHard3d, tauHard, NHard)

In [ ]:
offset = 39//2
height, width = im.shape
nu = nu_basic[offset:offset+height, offset:offset+width]
delta = delta_basic[offset:offset+height, offset:offset+width]
np.count_nonzero(np.isnan(delta))
delta.shape

In [ ]:
plt.imshow(basic_estimate, cmap='gray')

In [ ]:
%%time
final_estimate = bm3d_2nd_step(imbr, basic_estimate, sigma, kWien, nWien, lambdaHard2d, lambdaHard3d, tauWien, NWien)

In [ ]:
plt.imshow(final_estimate, cmap='gray')

### PSNR: Peak Signal to Noise Ratio

In [ ]:
def compute_rmse(reference_image, denoised_image):
    """
    Compute the Root Mean Square Error (RMSE) between two images.
    
    Parameters:
        reference_image (numpy.ndarray): The noiseless reference image (u_R).
        denoised_image (numpy.ndarray): The denoised image (u_D).
        
    Returns:
        float: The RMSE value.
    """ 
    # Compute RMSE using vectorized operations
    mse = np.mean((reference_image - denoised_image) ** 2)
    if mse == 0:
        return float('inf')  # PSNR is infinite if the images are identical
    rmse = np.sqrt(mse)
    
    return rmse

In [ ]:
def compute_rmse(reference_image, denoised_image):
    """
    Compute the Root Mean Square Error (RMSE) between two images, ignoring NaN values.
    
    Parameters:
        reference_image (numpy.ndarray): The noiseless reference image (u_R).
        denoised_image (numpy.ndarray): The denoised image (u_D).
        
    Returns:
        float: The RMSE value.
    """
    # Create a mask to ignore NaN values in either image
    valid_mask = ~np.isnan(reference_image) & ~np.isnan(denoised_image)
    
    # Apply the mask to select valid pixels only
    valid_reference = reference_image[valid_mask]
    valid_denoised = denoised_image[valid_mask]
    
    # Compute the Mean Squared Error (MSE) only over valid pixels
    mse = np.mean((valid_reference - valid_denoised) ** 2)
    
    # If MSE is zero, return infinity for RMSE to reflect perfect similarity
    if mse == 0:
        return float('inf')
    
    # Compute RMSE
    rmse = np.sqrt(mse)
    return rmse

In [ ]:
def compute_psnr(reference_image, denoised_image):
    """
    Compute the Peak Signal-to-Noise Ratio (PSNR) between two images.
    
    Parameters:
        reference_image (numpy.ndarray): The noiseless reference image.
        denoised_image (numpy.ndarray): The denoised image.
        
    Returns:
        float: The PSNR value in decibels (dB).
    """
    # Ensure the two images have the same shape
    assert reference_image.shape == denoised_image.shape, "Images must have the same dimensions"
    
    # Compute RMSE
    rmse = compute_rmse(reference_image, denoised_image)
    
    # Compute PSNR
    max_pixel_value = 255.0  # Assuming 8-bit images with pixel values in [0, 255]
    psnr = 20 * np.log10(max_pixel_value / rmse)
    
    return psnr

In [ ]:
compute_psnr(im, basic_estimate)

In [ ]:
compute_psnr(im, final_estimate)

In [ ]:
np.count_nonzero(np.isnan(basic_estimate))

subplot: original \ noisy (sigma) \ 1st step (psnr) \ 2nd step (psnr) \ tempo de execução

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_images_with_note_and_subtitles(images, titles, subtitles, note, figsize=(10, 8)):
    """
    Display 4 images in a 2x2 grid with titles, and subtitles for the last two images.
    Add a note in the bottom-right corner.
    
    Parameters:
        images (list of numpy.ndarray): List of 4 images to display.
        titles (list of str): List of 4 titles corresponding to the images.
        subtitles (list of str): List of 2 subtitles for the last two images (or empty strings if not needed).
        note (str): The note to display in the bottom-right corner (e.g., execution time).
        figsize (tuple): Size of the entire figure.
    
    Returns:
        None
    """
    # Check if input is valid
    if len(images) != 4 or len(titles) != 4 or len(subtitles) != 2:
        raise ValueError("You must provide exactly 4 images, 4 titles, and 2 subtitles.")
    
    # Create the figure and axes for the subplot
    fig, axes = plt.subplots(2, 2, figsize=figsize)
    axes = axes.flatten()  # Flatten the 2x2 grid into a 1D array for easier iteration
    
    # Plot each image with its corresponding title and subtitles (if applicable)
    for i, ax in enumerate(axes):
        ax.imshow(images[i], cmap='gray', vmin=0, vmax=255)  # Assuming grayscale images
        ax.set_title(titles[i], fontsize=12, pad=10)  # Title above the image
        ax.axis('off')  # Hide axes for a cleaner look
        
        # Add subtitle below the last two images
        if i >= 2:  # Indices 2 and 3 correspond to the last two images
            ax.text(
                0.5, -0.05, subtitles[i - 2],  # Subtitle positioned closer to the image
                ha='center', va='center', transform=ax.transAxes, fontsize=10, color='black'
            )
    
    # Adjust layout to make space for the note at the bottom
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.2)  # Add space at the bottom for the note
    
    # Add a small note in the bottom-right corner
    fig.text(0.95, 0.03, note, ha='right', fontsize=10, color='blue')  # Bottom-right corner note
    
    # Display the plot
    plt.show()

# Example usage
if __name__ == "__main__":
    # Create example grayscale images (random noise for demonstration purposes)
    images = [np.random.randint(0, 256, (100, 100), dtype=np.uint8) for _ in range(4)]
    titles = ["Image 1", "Image 2", "Image 3", "Image 4"]
    subtitles = ["Subtitle for Image 3", "Subtitle for Image 4"]
    note = "Execution Time: 1.23 seconds"
    
    # Call the function to plot images with titles, subtitles, and the note
    plot_images_with_note_and_subtitles(images, titles, subtitles, note)
